In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import scanpy as sc
import tifffile
import torch
import torchvision.transforms.functional as TF
import yaml
from einops import rearrange

In [3]:
import scanpy as sc
adata = sc.read_10x_mtx(
    '/diskmnt/Projects/Users/estorrs/single-cell-pdac/data/single_cell/original/inhouse/wu01/',
    var_names='gene_ids'
)
adata

/diskmnt/Projects/Users/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


AnnData object with n_obs × n_vars = 737280 × 33538
    var: 'gene_symbols', 'feature_types'

In [6]:
sc.__version__

'1.9.5'

In [7]:
! ls -l /diskmnt/Projects/Users/estorrs/single-cell-pdac/data/single_cell/original/inhouse/wu01/

total 15184
-rw-r-----. 1 estorrs users  2550196 Apr 28  2022 barcodes.tsv.gz
-rw-r-----. 1 estorrs users   304728 Apr 28  2022 features.tsv.gz
-rw-r-----. 1 estorrs users 12686228 Apr 28  2022 matrix.mtx.gz
